# Exploring conformational space of selected macrocycles - notes

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [147]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [148]:
import glob
import py3Dmol

## "M1" macrocycle

The crystallographic structure of "M1" (shown below) was used as a starting geometry for generating various conformers.

In [149]:
cm1 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.xyz','r').read()
vcm1 = py3Dmol.view(width=400,height=400)
vcm1.addModel(cm1,'xyz')
vcm1.setStyle({'stick':{}})
vcm1.setBackgroundColor('0xeeeeee')
vcm1.zoomTo()
vcm1.show()

### Conformers generated with the Balloon software:

In [150]:
inps = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_crystal/*.sdf')
n = len(inps)
n

6

In [151]:
# grep energies
e_m1 = []
for inp in inps:
    with open(inp,'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if "M  END" in line:
                e_m1.append(float(lines[i+1]))
e_m1

[53.24562081155721,
 53.58376642026054,
 57.7600582967278,
 62.42446771209878,
 55.44127295167746,
 53.65138540775401]

Conformers (from the lowest energy):

In [152]:
%%html
<table>
  <tr>
    <td id="m1_b1" ></td>
    <td id="m1_b2" ></td>
    <td id="m1_b3" ></td>   
  </tr>
  <tr>
    <td> m1_b1, E = </td>
    <td> m1_b2, E = </td>
    <td> m1_b3, E = </td>   
  </tr>
  <tr>
    <td id="m1_b4" ></td>    
    <td id="m1_b5" ></td>
    <td id="m1_b6" ></td>   
  </tr>
  <tr>
    <td> m1_b4, E = </td>
    <td> m1_b5, E = </td>
    <td> m1_b6, E = </td>   
  </tr>
</table>

,,
"m1_b1, E =","m1_b2, E =","m1_b3, E ="
,,
"m1_b4, E =","m1_b5, E =","m1_b6, E ="


In [153]:
p1_b_handles=[]
for inp in inps:
    f1_b = open(inp, 'r').read()
    p1_b = py3Dmol.view(width=200,height=200)
    p1_b.addModel(f1_b,'sdf')
    p1_b.setStyle({'stick':{}})
    p1_b.setBackgroundColor('0xeeeeee')
    p1_b.zoomTo()
    p1_b_handles.append(p1_b)

In [154]:
p1_b_handles[0].insert('m1_b1')

In [155]:
p1_b_handles[1].insert('m1_b2')

In [156]:
p1_b_handles[2].insert('m1_b3')

In [157]:
p1_b_handles[3].insert('m1_b4')

In [158]:
p1_b_handles[4].insert('m1_b5')

In [159]:
p1_b_handles[5].insert('m1_b6')

### Conformers generated with the RDKit software:

In [160]:
inps = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_sdf/*.sdf')
n = len(inps)
print(n)

12


In [161]:
%%html
<table>
  <tr>
    <td id="a1" ></td>
    <td id="a2" ></td>
  </tr>
  <tr>
    <td> p1 </td>
    <td> p2 </td>
  </tr>
</table>

,
p1,p2


In [162]:
m1 = open(inps[1], "r").read()
p1 = py3Dmol.view(width=200,height=200)
p1.addModel(m1,'sdf')
p1.setStyle({'stick':{}})
p1.setBackgroundColor('0xeeeeee')
p1.zoomTo()
p1.insert('a1')

In [163]:
m2 = open(inps[3], "r").read()
p2 = py3Dmol.view(width=200,height=200)
p2.addModel(m1,'sdf')
p2.setStyle({'stick':{}})
p2.setBackgroundColor('0xeeeeee')
p2.zoomTo()
p2.insert('a2')

## "M7" macrocycle

start: crystal geometry

In [164]:
m7 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m7/m7_crystal.xyz','r').read()
p7 = py3Dmol.view(width=400,height=400)
p7.addModel(m7,'xyz')
p7.setStyle({'stick':{}})
p7.setBackgroundColor('0xeeeeee')
p7.zoomTo()
p7.show()

### Conformers generated with the Balloon software:

In [165]:
inps = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/results_crystal/*.sdf')
n = len(inps)
print(n)

10


In [166]:
m1 = open(inps[2], "r").read()
p1 = py3Dmol.view(width=400,height=400)
p1.addModel(m1,'sdf')
p1.setStyle({'stick':{}})
p1.setBackgroundColor('0xeeeeee')
p1.zoomTo()
p1.show()